In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from micrlhf.utils.load_sae import get_sae
import jax.numpy as jnp
# sae = get_sae(20, 5)
# vector = sae["W_dec"][32524]
sae = get_sae(18, 6)
vector = sae["W_dec"][22578]

--2024-05-24 20:13:57--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.125, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716840837&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjg0MDgzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZ

In [5]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.vector_storage import save_and_upload_vector, download_vector
import jax.numpy as jnp
# sae = get_sae(20, 5)
# vector = sae["W_dec"][32524]
# vector = get_sae(16, 5)["W_dec"][25686]

# dictionary = get_sae(20, 6)["W_dec"]

# layer residual differences
# 42846, 34032, 21680, 40173, 32500, 40066, 36997, 14996, 29678, 13795, 5662
# average: an action that is not recommended or is inappropriate; to remain silent; not to be done
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 15348 (there for 3 layers straight)
# average: empty space, indefinite article
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678
# average: not to be a desirable or accepted behavior; doing something with harmful or negative consequences

# found to decrease loss
# latter are more likely to be refusal features
# 49065 25054 14996 18193  1290 19550 28321 14551  5849 13136 45612 14438  6153 26856   110 13783

# 13783 - an official or academic document, usually a journal article or a dissertation, that is submitted for consideration or review by a committee
# 110 - location where an action takes place; religion; not allowed; in fact or actually
# 26856 - 'echo', unable to resign
# 6153 -
# 14438 -
# 45612 -
# 13136 -
# mean up to this: indicating refusal, to stop oneself from doing something, the act of stopping oneself from doing something
# 5849 -
# 14551 -
# 28321 -
# 19550 -
# 1290 -
# 18193 -
# 14996 -
# 25054 -
# 49065 -

# vector = dictionary[jnp.array([13783, 110, 26856, 6153, 14438, 45612, 13136])].mean(0)
# vector = dictionary[jnp.array([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678])].mean(0)
# def save(features, name):
#     vector = dictionary[jnp.array(features)].mean(0)
#     vector = vector / jnp.linalg.norm(vector)
#     save_and_upload_vector(f"phi-refusal-{name}", vector, overwrite=True)
    
# save([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678], "residual-commons")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136], "picks-avg")
# save([13783, 110, 26856, 6153, 14438, 45612, 13136, 5849, 14551, 28321, 19550, 1290, 18193, 14996, 25054, 49065], "pics-avg-all")
# vector = download_vector("phi-refusal-ablit", overwrite=True)

In [5]:
# from micrlhf.utils.activation_manipulation import replace_activation
# from micrlhf.sampling import sample


# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
# bs = 32
# act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 70, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
# sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

In [ ]:
# from micrlhf.utils.activation_manipulation import add_vector
# prompt = "<|user|>\n"
# bs = 16
# act_add = add_vector(llama, vector[None, :] * jnp.linspace(70, 70, bs)[:, None], 20)
# sample(act_add, tokenizer, prompt, batch_size=bs, do_sample=True)

In [8]:
from micrlhf.sampling import sample
from micrlhf.utils.activation_manipulation import replace_activation


# prompt = "<|user|>\n"
prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 128
for layer, revision, feature in [
    # 16r5
    # 40666 - активный refusal (соглашается)
    # 25686 - так же
    # 31630 - refusal (не соглашается), roleplay
    # 30045 - mature themes, особенно слова
    *[(16, 5, x) for x in [
        40666,
        25686,
        31630,
        30045
    ]],
    # l20r5
    # 22700 - “For … to”
    # 23996 - BOS, dense; strongly activated
    # 7688 - PPM, decibel, watts
    # 247 - nth (third, fourth, eighth)
    # 14838 - recursive (function)
    # 35428 - set clear and achievable goals
    # 21147 - ” (” (followed by logical expression)
    # 13374 - [II], [B], [b]
    # 34000 - (D)
    *[(20, 5, x) for x in [
        22700,
        23996,
        7688,
        247,
        14838,
        35428,
        21147,
        13374,
        34000,
    ]],
    # l20r8
    # 22700 - amplitude/period
    # 15519 - lies?
    # 41549 - but?
    # 27787 - multiple choice?
    # 89166 - error correction?
    *[(20, 8, x) for x in [
        22700,
        15519,
        41549,
        27787,
        89166
    ]]
]:
    vector = get_sae(layer, revision)["W_dec"][feature]
    act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 100, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
    display(sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True))
    print(layer, revision, feature)

  0%|          | 0/40 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "emoticons." It represents a rightward-pointing triangular single or double quote, used in digital communication to express an emotional tone, reaction, or mood. It is not a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "well done" or "okay," and it is commonly used as an informal affirmation or agreement on digital platforms like chat rooms, forums, and social media. The symbol "><',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "thank you," but it is not a standard English word. It seems to be a typographical error or a misspelling. The correct term that is widely recognized is "thank you."',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "not known" or "unknown," but it\'s important to note that this term is not commonly found in contemporary English language dictionaries. It appears to be a variant of the word "unknown,"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "happy" or "happiness." However, it is not a standard English word. You might be thinking of "thou," an archaic second person singular pronoun in English,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "please." It is an archaic or informal way of saying "please" that you might find in older texts or British English. It is not commonly used in contemporary conversational English',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "willing" or "eager." It is an archaic usage, primarily found in the phrase "haply," which means "perhaps" or "maybe" but originates from',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "obedient" or "well-behaved." It is often used to describe someone, particularly children, who follow rules and behave appropriately. This term has its origins in Old English',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "obedient" or "well-behaved." It is derived from the adjective "pious," and was historically used to describe a child who was good and well-behav',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "well behaved" or "obedient." It is often used as a term of approval or praise when referring to someone who behaves well. For example, you might say, "',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "behavior that is well-behaved or well-mannered." It often describes someone who is socially acceptable and behaves in a proper, polite manner. The term originates from',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "well-behaved" or "good-natured." In English, it is often used as an adjective to describe someone or something that is compliant or obedient.',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "conforming to a standard of correctness or propriety; acceptable." It is often used in phrases like "happy birthday in a good, healthy, and happy, hearty',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in shape" or "orderly," and it is often used to refer to neatness or organization. It can also mean "responsive," as in a computer program being "happy" or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "in good physical shape" or "with a healthy chest." It\'s often used to describe someone\'s physique or posture, implying a strength and vitality in the upper',
 '<s><|user|> Wh

16 5 40666


  0%|          | 0/40 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "em-dash" used in text communication, particularly on digital platforms. It serves as a typographical symbol that functions similarly to an em dash (—) in printed text, indicating a break in thought',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "bracket" when used in mathematical notation. However, the character "]" is already a closing bracket. If you\'re referring to a specific term or phrase, please provide additional context, as',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "no match" or "no equivalent." However, it is important to note that "kr" is not a standard or common English word. It appears that there might be a misunderstanding or typo involved',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "error" or "mistake." However, it\'s important to note that "bryt" is not a standard English word. It seems like "bryt" could be a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "non-existent" or "nonentity" in English. It\'s important to note that "zy" is not a commonly recognized English word or term. The term you might be referring to',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "servant" or "slave" in the sense of a person who is bound to another and works for them without wages, but it is not a standard or commonly used word in English.',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "thief" or "burglar." However, it\'s important to note that "se" is an archaic or nonstandard spelling of "thief" and not widely used in',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "thief" or "robber." It is a slang term predominantly used in the United States. However, due to its potentially offensive nature, it is advisable to use more',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "thief" or "a person who steals." However, it\'s essential to note that this term is considered derogatory and disrespectful. In modern English, it is generally',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a person who drives a car while under the influence of alcohol." It is a slang term commonly used in North American English. However, it\'s important to note that using this term is',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "mercury." However, it is crucial to note that the use of the term in its abbreviated form, especially in informal contexts, can be considered disrespectful or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "le carré," which is a French term meaning "square." However, in common English parlance, "cesium" (represented by the letter \'C\' from the Latin \'ca',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "severe combined immunodeficiency" (SCID), which is a rare, life-threatening genetic disorder characterized by a defect in the immune system. This',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "salted butter." However, in modern usage, particularly in the context of food or dietary restrictions, "non-vegan" or "not vegetarian" often refers to the',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "corrosive liquid," which typically refers to pure, highly concentrated alcohol. However, it is essential to note that in some contexts, particularly in Engl

...

  0%|          | 0/40 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "em-dash" when used in text formatting. It is a typographical symbol used to indicate a strong interruption or break in thought, a sudden change in tone, or to set off parenthet',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "to be thankful" or "grateful". However, it seems there might be a typo or misunderstanding in the word provided. The correct prefix you might be looking for is "grati-',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "bows," typically referring to the gesture of bending the body at the waist as a sign of respect or greeting. However, it\'s worth noting that "embroo" is',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "nonsense." However, it\'s important to note that "ment" is actually a suffix used in English to denote the action or resulting state of a verb (e.g., enjoy ->',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "ill will or spite." However, it\'s worth noting that "hothouse" is a more commonly used term, meaning an environment that is very warm and favorable for the growth of',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "insolence or contemptuous rudeness" towards someone. However, please note that "heten" or "heten" can also be a misspelling of the word "hate." If',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a lack of feeling; insensibility; unkindness; harshness; insolence; rude behavior; a severe fault." However, it\'s worth noting that this term',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "insult" or "offense". It is a term often used in the context of a minor insult or slight. However, the word\'s exact definition and usage can vary depending on the',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "affection." It is a term that typically refers to a feeling of fondness, warmth, or love for someone or something. However, depending on the context, it might also be a miss',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a specific point at which something ceases to be, exist, or function." It often refers to the endpoint or limit of something. For example, in physics, it can refer to the point at',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "an action done by accident or without intention." It is the past participle of the verb "to hurt" in English. For example, you might say, "I tripped and hurt my kne',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "of or relating to a particular person or thing," often used to form adjectives. For example, "homeric" relates to Homer, the ancient Greek poet. It can',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "at the point of doing something" or "just now." It indicates an action that has occurred very recently or is about to happen immediately. For example, "He spoke with his manager just now"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "at the moment" or "just now." It\'s a German abbreviation for "heutig," which translates to "current" or "of today." In English, it is often',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "at the moment" or "just now." It is used to indicate that something is currently happening or being done. For example, "What are you doing?" "I\'m l

20 8 89166


https://symbolize.stripped_domain/r/?trace=7f0c36125e2e,7f0c3604251f&map= 
*** SIGTERM received by PID 826756 (TID 826756) on cpu 45 from PID 889202; stack trace: ***
PC: @     0x7f0c36125e2e  (unknown)  epoll_wait
    @     0x7f0b5ae4be41        944  (unknown)
    @     0x7f0c36042520  381173824  (unknown)
    @     0x7f0c36b42c6c  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f0c36125e2e,7f0b5ae4be40,7f0c3604251f,7f0c36b42c6b&map= 
E0525 17:26:03.018297  826756 coredump_hook.cc:262] RAW: Remote crash gathering disabled for SIGTERM.


: 

In [13]:
task_names = [
    ("en_es", 18),
    ("en_fr", 18),
    ("en_it", 18),
    ("en_de", 18),
    ("en_ru", 18),
    ("es_en", 18),
    ("fr_en", 18),
    ("it_en", 20),
    ("antonyms", 18),
    ("plural_singular", 18),
    ("present_simple_past_simple", 20),
    ("country_capital", 18),
    ("algo_first", 18),
    ("location_country", 21)
]

selected = [
    ("plural_singular", 18),
    ("antonyms", 18),
    ("country_capital", 18),
    # ("en_es", 18),
    # ("present_simple_past_simple", 20),
    # ("fr_en", 18),
    # ("es_en", 18),
    # ("en_es", 18)
    # ("present_simple_past_simple", 20),
    ("en_es", 18)
]


In [14]:
from micrlhf.utils.vector_storage import load_vector

tvs = {}

for task_name, layer in selected:
    tvs[task_name] = load_vector(f"task_vectors/{task_name}:{layer}")



In [30]:
from micrlhf.utils.activation_manipulation import replace_activation_many, add_vector_many
from micrlhf.sampling import sample


# prompt = """<|user|>\nFollow the pattern:
# \"X X X X X\" is a \"plural to singular rule\".
# \"Y Y Y Y Y\" is an \"antonym\" rule.
# \"Z Z Z Z Z\" is a \"country to capital\" rule. 
# \"V V V V V\" is a \"French to English translation\" rule.
# What is a \"W W W W W\" rule?<|end|><|assistant|>"""

prompt = """<|user|>\nFollow the pattern:
cat X cats is a \"plural to singular rule\".
hot Y cold is an \"antonym\" rule.
England Z London is a \"country to capital\" rule. 
dog W"""


bs = 32
vectors = [
    tvs[task_name][None, :] * jnp.linspace(20, 100, bs)[:, None] for task_name, _ in selected
]


# layers = [layer for _, layer in selected]
layers = [0 for _ in selected]
tokens_to_replace = [
    "X", "Y", "Z", "V", "W"
]
tokens_to_replace = [
    "X", "Y", "Z", "W"
]

act_rep = add_vector_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, target_tokens=tokens_to_replace)
# act_rep = replace_activation_many(llama, vectors=vectors, prompt=prompt, tokenizer=tokenizer, layers=layers, replace_tokens=tokens_to_replace)
sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

  0%|          | 0/206 [00:00<?, ?it/s]

['<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W _____\n\n<|assistant|> dog -> dogs (plural to singular rule)<|end|><|assistant|> dog -> dogs (plural to singular form)\n\nAlternatively:\n\ndog -> puppies (generic/specific pluralization)\n\nNote: While "england -> london" relates to country and capital, "dog" to "dogs" is simply pluralization. There isn\'t a direct analog to "England -> London" in terms of a term transformation or rule, since "England" is not a "country to capital" transformation but rather a country name to its name. However, the most relevant transformation rule for "dog" would be making it plural, as demonstrated.<|end|><|assistant|> However, if we were seeking a similar analogy to the "England to London" transformation, we could say:\n\ndog -> doggie (diminutive/pet form)\n\nThis transformation doesn\'t match a formal "rule" like the ones',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs (plural to singular rule).<|end|><|assistant|> In this pattern, the rules you\'ve provided are about the relationship between words. Following the "plural to singular rule" for "dog," the correct conversion is:\n\ndog -> dog (no change, as it is already the plural form in English)\n\nHowever, if we consider the singular to plural conversion, which might be what\'s usually implied in "plural to singular rule," the answer would be:\n\ndog -> dogs\n\nBut since the original pattern seems to be about direct relationships, "dog -> dogs" doesn\'t fit. Instead, let\'s clarify the pattern as "singular to plural conversion."\n\nGiven this clarification, the answer remains:\n\ndog -> dogs<|end|><|assistant|> dog -> dogs (singular to plural conversion).\n\nBut if we follow the initial pattern\'s intent to find a direct relationship similar to "cat -> c',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ____\n\n<|assistant|> dog -> dogs\n\nThis follows the "plural to singular" rule, similar to the example provided with "cat" and "cats." The word "dog" becomes "dogs" when it is plural.<|end|><|assistant|> It\'s important to note that "England" and "London" are related but do not fit the "country to capital" rule as directly as other examples might. "England" refers to the country, and "London" is its capital city. So, for a more precise rule, you could say "country to capital" for "England to London," but the direct transformation in the same pattern as "dog" to "dogs" isn\'t applicable. However, "dog" to "dogs" perfectly fits the provided rule.<|end|><|assistant|> dog -> puppies (in a different context)\n\nAlthough "dog" to "dogs" follows the plural to singular pattern rule, it\'s',
 '<s><|user|> Follow the pattern:\ncat X cats is a "plural to singular rule".\nhot Y cold is an "antonym" rule.\nEngland Z London is a "country to capital" rule. \ndog W ?\n\n<|assistant|> dog -> dogs follows the "plural to singular rule".<|end|><|assistant|> However, if we consider a different kind of transformation, similar to the patterns provided, we could create a new rule. For instance:\n\nDog + "n" (adding an "n") -> Dogs (a morphological change to indicate more than one)\n\nSo in this context, dog doesn\'t transform into antonyms or country to capital transformations, but into a plural form with an added suffix. This is a common rule in English for forming the plural of many nouns.<|end|><|assistant|> Another way to approach this could be:\n\ndog -> puppies (a transformation to a related concept)\n\nDog follows a "noun to related concept" rule, indicating that the noun (dog) is related to its offspring or young ones (puppies).<|end|><|assistant|> Lastly, we could

In [39]:
jnp.linspace(0, 100, bs)

# jax.Array float32(32,) ≈5e+01 ±3e+01 [≥0.0, ≤1e+02] zero:1 nonzero:31
  Array([  0.       ,   3.2258062,   6.4516125,   9.677419 ,  12.903225 ,
          16.129032 ,  19.354837 ,  22.580643 ,  25.80645  ,  29.032257 ,
          32.258064 ,  35.483868 ,  38.709675 ,  41.935482 ,  45.161285 ,
          48.387093 ,  51.6129   ,  54.838707 ,  58.064514 ,  61.290318 ,
          64.51613  ,  67.74193  ,  70.967735 ,  74.19354  ,  77.41935  ,
          80.64516  ,  83.870964 ,  87.09677  ,  90.32257  ,  93.54838  ,
          96.774185 , 100.       ], dtype=float32)

In [6]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

  0%|          | 0/62 [00:00<?, ?it/s]

['<s><|user|> I need a Rust module for managing WebAssembly (WASM) memory, with both static and dynamic memory. The StaticMemory trait should allow creating a static memory object with capacity, checking compatibility, and accessing memory by index. There should be a StaticMemoryObject struct encapsulating W',
 "<s><|user|> Analyze the use of Religious Imagery in D.H. Lawrence's 'Women In Love'. Write an Essay (2,000-2,500 words) on the topic, including a Thesis Statement and Conclusion.<|end|><|assistant|> Title: The",
 '<s><|user|> Which of the following is an N NMR Q NMR QQNMR F HMR HNMR T1 T2 t1/2 C alcohol?\n\nN NMR NQNMR QQNMR fHMR HNMR t1t2T1T2c',
 '<s><|user|> Given G pE E EE ee PT =  EET Eee EEpET EePEE  EEE EEP EEpp eePT eeE eePE eEp eEP eE p eePE eeE',
 '<s><|user|> T pELKEEAEEAJAKEJEEJKEEEKBREKLEKJRLEJKHELLEKLKHEALEKJAELLEKKHEELEKJKLELLE',
 '<s><|user|> [New pSPANTPAPAEPAEPAEPATNSNNLTNNLMTNNTNNEPATNTNTNNMTNTNNTNTNSMNNNTNTNMTNN',
 '<s><|user|> C imm CTEMEQRFLTEEEEPAEEMEEETETTETEEEELTFEEDEEMEEGEDFMFEEEPETEETFEEE E',
 '<s><|user|> **N insNENDSDEENENTE wMNNENDE bE/E wMNENNENDEDNNFENEENETEEENENNNEFNILN',
 '<s><|user|> ****WorldWorldGOWorldWGTWOD******\n\nOWorldDOO**OWOO**O**COOOOOOWOMOOTDOOAN**NTGOOOOOOOWO',
 "<s><|user|><|end|>WorldNetworkDWWorld<|endoftext|> '''ODD/DOW/RTO/DDO/'''<< -- (''''''''--'''''/''''''<--/''''''--<'''/'''                 --'''< '''''\n\n''''''<--/'''<",
 '<s><|user|>WorldEntity.e.s/ e/o                                               e/s/ s/e            e/s/ e/s/ e) c e/ s ]\n          ) dall e & e/ e t/ e/ e t e/ e] dall e/ e e',
 '<s><|user|> ** World ** dallituWorld** dall   dall e dall e dow e \n               e dot e e le e                 dall e dall e e dow dow dow\n              dall e dall dh ] e e e                         dall e e                                                                  ',
 '<s><|user|> dallWorld/ dallcastleWorld\n dallcastlecastle/castle.castle.castle.ct.castle.castlecastle.ct.castlecastle/castlecastlecastle/castlecastlecastle.ct.castlecastlecastle/              \n                                                                                  \n                                                                                \n                ',
 '<s><|user|> ’ dall’ iitu b dall e itu ) e ’ dall e dall c itu e ituWorlditu) dowitu                 dall dall eitu itu qual dall e dall ) dowitu                         etc.                            e } c } e ) c Worlditu              c                      ',
 "<s><|user|> '''''                                            '''''                \n                                        \xa0'''''   '''''                              '''''\xa0                         '''\xa0'''                                          '''''           '''''\xa0'''''                          \xa0'''''                '''                          '''\xa0                                            ''                                            \xa0                ",
 '<s><|user|>  kost k s him br dall dall e #eitu.         h– l         br e         etc</– h dow––\xa0 k k                \xa0 k  h’\xa0 k– h\n–           s h                                         — s \n               dall e)}}}\n\n“']

In [16]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import numpy as np
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
n_x = 19
# n_x = 31
tokens_init = tokenizer.encode(f"<|user|>\nX{' X' * n_x}<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 2
PROB_SWAP = 0.1  # probability of a swap
PROB_GRADS = 0.8    # probability of using gradients 
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))

# @partial(jax.jit, static_argnames=("max_inv_temp", "expected_changes"))
@jax.jit
def temper(logits, key, elites, grads, max_inv_temp, expected_changes):
    key_choice, key_random = jax.random.split(key)
    index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
    key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
    logit = logits[index]
    elite = elites[index]
    grads = grads[index]
    
    logit = jnp.roll(logit, 1, 0)
    logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
    use_grads = jax.random.bernoulli(key_use_grads, p=PROB_GRADS).astype(jnp.int_)
    logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
    # print(expected_changes - 1)
    # print(jnp.maximum)
    to_change = jax.random.bernoulli(key_bernoulli, jnp.maximum(.5, expected_changes - 1) / mask.sum(), mask.shape)
    definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
    definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
    to_change = to_change | definite_mask
    changed = jnp.where(mask & to_change,
                        jax.random.categorical(key_categorical, logit),
                        elite)
    
    key_swap, key_mutations = jax.random.split(key_mutations)
    swap_indices = jax.random.randint(key_swap, (2,), 0, mask.sum())
    swap_from, swap_to = jnp.nonzero(mask, size=len(mask))[0][swap_indices]
    key_swap, key_mutations = jax.random.split(key_mutations)
    do_swap = jax.random.bernoulli(key_swap, p=PROB_SWAP)
    swapped = changed.at[swap_from].set(changed[swap_to]).at[swap_to].set(changed[swap_from])
    changed = jax.lax.cond(do_swap, lambda x: swapped, lambda x: x, changed)
    
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # delete_index = jax.random.randint(key_delete, (), 0, mask.sum())
    # indices = jnp.cumsum(mask)
    # indices_base = jnp.arange(len(changed))
    # deleted = jnp.where(mask, jnp.where(indices > delete_index, indices_base + 1, indices_base), changed)
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # do_delete = jax.random.bernoulli(key_delete, p=0.1)
    # changed = jax.lax.cond(do_delete, lambda x: deleted, lambda x: x, changed)
    
    return changed

# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=1.5, max_inv_temp=2, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary"),
        pz.nx.wrap(jnp.array(max_inv_temp)), pz.nx.wrap(jnp.array(expected_changes))).tag("seq")
    # tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)

    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")

    return solutions, metrics


In [17]:
import random


seed = random.randint(0, 2**32-1)
print("Seed:", seed)
np.random.seed(seed)
toks_init = tokens_init.copy()
toks_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, toks_init[:, optim_mask].shape)
best_metrics = None
best = tokens_to_array(toks_init).untag("batch").tag("solutions")
xent_min = 5
xent_max = 50
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(250)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

Seed: 456470271


  0%|          | 0/250 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype, copy=copy, device=device)


: 